In [ ]:
import json
import hmac
import time
import base64
import urllib3
import hashlib
import requests
import pandas as pd
from requests.auth import AuthBase

In [ ]:
api_cred = pd.read_json("../API_KEYS/api_keys.json", orient= "index")
api_url = pd.read_json("../API_KEYS/api_url.json", orient= "index")

In [ ]:
api_url_cnbs = api_url.loc["CNBS"].URL
api_url_cnbs

In [ ]:
time.strftime("%Y-%m-%d")

In [ ]:
# Create custom authentication for Exchange
class CNBSExchangeAuth(AuthBase):
    def __init__(self, exchange):
        self.api_key = api_cred.loc[exchange].KEY
        self.secret_key = api_cred.loc[exchange].SECRET

    def __call__(self, request):
        timestamp = str(time.time())
        hmac_key = base64.b64decode(self.secret_key)
        message = timestamp + request.method + request.path_url + (request.body or '')
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        digest = signature.digest()
        signature_b64 = base64.b64encode(digest).decode('utf-8').rstrip('\n')
        
        current_date_version = (time.strftime("%Y-%m-%d"))

        request.headers.update({
          'CB-ACCESS-SIGN': signature_b64,
          'CB-ACCESS-TIMESTAMP': timestamp,
          'CB-ACCESS-KEY': self.api_key,
          'CB-VERSION': '2017-09-01',
          'Content-type': 'application/json' # <- this was missing
        })
        return request

In [ ]:
auth = CNBSExchangeAuth(exchange="CNBS")
auth

In [ ]:
# Get accounts
balance_data = requests.get(api_url_cnbs + 'user', auth=auth)
print(balance_data)
balance_data = balance_data.json()
# balance_data = pd.DataFrame(balance_data)
# balance_data.drop("profile_id", axis=1, inplace=True)
# balance_data["product_id"] = balance_data.currency+"-USD"
# balance_data.balance = pd.to_numeric(balance_data.balance)
balance_data